In [57]:
from bs4 import BeautifulSoup
from requests import get
import csv
import re

In [73]:
url = 'https://www.carrefour.pl/artykuly-spozywcze'
food_list = []

def csv_price_format(price):
    return float(price.replace(" ", "").replace("zł", "").replace(",","."))


def csv_pname_format(product):
    return product.replace(",",".")

def product_data_tocsv(page_num):

    site = get(f'{url}?page={page_num}')
    bs = BeautifulSoup(site.content, 'html.parser')
    
    for offer in bs.find_all('div', class_='jss304'):
    
        try:
            
            product = csv_pname_format(offer.find('span',class_='MuiTypography-root jss199 jss310 MuiTypography-h5').get_text().strip())

            price = csv_price_format(offer.find('div', class_='MuiTypography-root jss318 MuiTypography-h3').get_text().strip())
            
            measure = offer.find('span',class_='MuiTypography-root jss199 jss321 MuiTypography-caption MuiTypography-colorTextSecondary').get_text().strip()
            
            if re.search(r'kg\Z', measure):
                weight = float(re.split(r' kg\Z', measure)[0].replace(",","."))
                quantity = 'N/D'
                liters = 'N/D'

            elif re.search(r'szt.\Z', measure):
                weight = 'N/D'
                quantity = float(re.split(r' szt.\Z', measure)[0].replace(",","."))
                liters = 'N/D'

            elif re.search(r'l\Z', measure):
                weight = 'N/D'
                quantity = 'N/D'
                liters = float(re.split(r' l\Z', measure)[0].replace(",","."))
            
            
            price_kg_raw = offer.find('span',class_='MuiTypography-root MuiTypography-caption MuiTypography-colorTextSecondary').get_text().strip()
            

            if re.search(r'zł/1 kg\Z', price_kg_raw):
                price_kg = float(re.split(r'zł/1 kg\Z', price_kg_raw)[0].replace(",","."))

            elif re.search(r'szt.\Z', price_kg_raw):
                price_kg = float(re.split(r' zł/1 szt.\Z', price_kg_raw)[0].replace(",","."))

            elif re.search(r'l\Z', price_kg_raw):
                price_kg = float(re.split(r' zł/1 l\Z', price_kg_raw)[0].replace(",","."))


                
            food_list.append([product,price,price_kg,weight,quantity,liters])
            
        except AttributeError:
            pass
# for i in range(23,25):        
product_data_tocsv(23)
    
with open('food_item_list.csv', 'w', encoding='utf-8', newline='') as food_csv:
    writer=csv.writer(food_csv)
    writer.writerow(['Produkt','Cena','Cena za kg/szt./litr','Waga [KG]','Ilość [Sztuki]','Litry'])
    writer.writerows(food_list)
        

ValueError: could not convert string to float: '9\xa0795.00 '